# Preliminary Operations
The same imports and auxiliary code is defined below:

In [10]:
from google.colab import drive
import os
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import random as rn
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from tensorflow.keras.applications.vgg16 import preprocess_input

drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
#ROOT_DIR = "/content/drive/Shareddrives/Giaquinta_Pasqualetti/"
ROOT_DIR = "/content/sample_data/content/drive/Shareddrives/Giaquinta_Pasqualetti/"

IMAGES_DIR = os.path.join(ROOT_DIR, "Data")
TRAIN_DIR = os.path.join(IMAGES_DIR, "Train")
TEST_DIR = os.path.join(IMAGES_DIR, "Test")

IMAGE_DIM = 64
RAN_SEED = 10024062

VALIDATION_SPLIT = 0.1
TRAIN_SPLIT = 0.2

NOTE: Interactions with Google Drive are commented out. For our own testing, we used files stored locally in the session to **significantly** speed up the computation. To run this notebook using files stored in Google Drive, uncomment the lines that are currently commented and comment out the lines that are using local files.

In [26]:
! rm -r /content/drive/Shareddrives/Giaquinta_Pasqualetti/Data
! unzip -q /content/drive/Shareddrives/Giaquinta_Pasqualetti/Data.zip -d /

#! rm -r /content/sample_data/*
#! unzip -q /content/Data.zip -d /content/sample_data/

rm: cannot remove '/content/drive/Shareddrives/Giaquinta_Pasqualetti/Data': No such file or directory
replace /content/drive/Shareddrives/DeepLearning/Data/Test/Forest/Forest_520.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace /content/drive/Shareddrives/DeepLearning/Data/Test/Forest/Forest_520.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [27]:
# Set all the seeds, to create datasets and Tensors from the same starting point
def set_seed():
    os.environ["PYTHONHASHSEED"]="0"
    np.random.seed(RAN_SEED)
    rn.seed(RAN_SEED)
    tf.random.set_seed(RAN_SEED)

# Returns Train, Val and Test sets
# BATCH_SIZE is user-given input, VALIDATION_SPLIT is 0.1
def load_datasets(BATCH_SIZE):
    set_seed()

    train = tf.keras.preprocessing.image_dataset_from_directory(
        TRAIN_DIR, labels='inferred', label_mode='categorical', class_names=None,
        color_mode='rgb', batch_size=BATCH_SIZE, shuffle=True, seed=RAN_SEED,
        validation_split=VALIDATION_SPLIT, subset='training', follow_links=False,
        image_size=(64,64)
    )

    val = tf.keras.preprocessing.image_dataset_from_directory(
        TRAIN_DIR, labels='inferred', label_mode='categorical', class_names=None,
        color_mode='rgb', batch_size=BATCH_SIZE, shuffle=True, seed=RAN_SEED,
        validation_split=VALIDATION_SPLIT, subset='validation', follow_links=False,
        image_size=(64,64)
    )

    test = tf.keras.preprocessing.image_dataset_from_directory(
        TEST_DIR, labels='inferred', label_mode='categorical',
        class_names=None, color_mode='rgb', batch_size=BATCH_SIZE, shuffle=True,
        seed=RAN_SEED, follow_links=False, image_size=(64,64)
    )

    return train, val, test

# Returns some details about trained model
def train_performance(history):

  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

# Test
def show_res(model, test):
  data, labels = [], []

  for data_batch, labels_batch in test:
    data.append(data_batch)
    labels.append(labels_batch)

  data = tf.concat(data, axis=0)
  labels = tf.concat(labels, axis=0)

  y_score = model.predict(data)

  y_pred = np.rint(y_score)

  y_true = np.asarray(labels)

  y_pred = np.argmax(y_score, axis=1)
  y_test = np.argmax(y_true, axis=1)

  print("Classification report: ")
  print(metrics.classification_report(y_test,y_pred ,digits = 4))


  cm = metrics.confusion_matrix(y_test, y_pred)
  disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm)
  disp.plot()
  plt.show()


# Function to plot multi-class ROC curves
def plot_multiclass_roc(model, test_data):
    test_images, test_labels = [], []

    for data_batch, labels_batch in test_data:
        test_images.append(data_batch)
        test_labels.append(labels_batch)

    test_images = np.concatenate(test_images, axis=0)
    test_labels = np.concatenate(test_labels, axis=0)

    # Preprocess images
    test_images = preprocess_input(test_images)

    # Get predicted scores for all classes
    y_scores = model.predict(test_images)

    # Binarize labels
    y_true = label_binarize(np.argmax(test_labels, axis=1), classes=range(10))

    # Compute ROC curve and AUC for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(10):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_scores[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Plot all ROC curves in a single plot
    plt.figure(figsize=(10, 8))
    for i in range(10):
        plt.plot(fpr[i], tpr[i], label='Class %d (AUC = %0.2f)' % (i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-Class ROC Curves')
    plt.legend(loc="lower right")
    plt.show()


def model_summary(model):
  from keras.utils.vis_utils import plot_model

  model.summary()
  plot_model(model, show_shapes=True, show_layer_names=True)


# VGG16 with Feature Extraction
Importing the **VGG16** Keras model. VGG16 is a deep neural network architecture known for its simplicity and effectiveness in image classification. It consists of 16 layers, mostly using 3x3 convolutional filters and 2x2 max-pooling layers. VGG16 captures features at different scales, making it useful for tasks like recognizing objects in images. Its pre-trained weights enable quick adaptation to new tasks, making it a popular choice for transfer learning.

Using VGG16 for satellite image land use classification is beneficial due to its hierarchical feature extraction, pre-trained weights, transfer learning potential, ability to handle large input sizes, and suitability for diverse landscapes. It offers good generalization and customization while aiding visual interpretation of features.

In [28]:
from tensorflow.keras.applications import VGG16

conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(64, 64, 3))

conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

Layer freezing is needed to fix the weights of certain neural network layers during training, commonly for transfer learning, to retain learned features and prevent them from being changed.

In [29]:
print('Number of trainable weights: ', sum(np.prod(x.shape) for x in conv_base.trainable_weights))
conv_base.trainable = False
print('Number of trainable weights after freezing the convolutional base:', sum(np.prod(x.shape) for x in conv_base.trainable_weights))

Number of trainable weights:  14714688
Number of trainable weights after freezing the convolutional base: 0


## **First try:** 256 neurons with adam optimizer
The conv_base initiates the feature extraction phase. As our parameters we decided to set the number of neurons in the dense layer to 256 and the number of neurons in the dense layer to 256, without any L2 regularization, dropout or data augmentation, which will be later added if necessary. The chosen optimizer is Adam.

In [30]:
inputs = keras.Input(shape=(64, 64, 3))
x = inputs
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 tf.__operators__.getitem_3  (None, 64, 64, 3)         0         
  (SlicingOpLambda)                                              
                                                                 
 tf.nn.bias_add_3 (TFOpLamb  (None, 64, 64, 3)         0         
 da)                                                             
                                                                 
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 2048)              0         
                                                                 
 dense_6 (Dense)             (None, 256)               5245

In [31]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
    ),
    keras.callbacks.ModelCheckpoint(
        #'/content/drive/Shareddrives/Giaquinta_Pasqualetti/Models/vgg16_simple.h5',
        '/content/vgg16_simple.h5',
        monitor='val_loss',
        mode='min',
        save_best_only=True,
    )
]

In [32]:
train, val, test = load_datasets(BATCH_SIZE = 64)
res = model.fit(train, epochs=15, validation_data=val, callbacks = callbacks_list)
train_performance(res)
show_res(model, test)
plot_multiclass_roc(model, test)

NotFoundError: ignored

## **Second try:** addressing the overfitting
The results indicate that the validation accuracy is lower than the training accuracy, and the validation loss is higher than the training loss by a significant margin. It is possible that the accuracy might not improve significantly or not improve at all. However, let's attempt to reduce overfitting so that the classification model is sufficiently generalized to consistently deliver this level of performance. To begin with, we will implement dropout and add a data augmentation layer.

Let's repeat the experiment:

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
])

inputs = keras.Input(shape=(64, 64, 3))
x = data_augmentation(inputs)
x = inputs
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
model.summary()

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
    ),
    keras.callbacks.ModelCheckpoint(
        #'/content/drive/Shareddrives/Giaquinta_Pasqualetti/Models/vgg16_dropout_augment.h5',
        '/content/vgg16_dropout_augment.h5',
        monitor='val_loss',
        mode='min',
        save_best_only=True,
    )
]

train, val, test = load_datasets(BATCH_SIZE = 64)
res = model.fit(train, epochs=15, validation_data=val, callbacks = callbacks_list)
train_performance(res)
show_res(model, test)
plot_multiclass_roc(model, test)

## **Third try:** Data regularization
Presently, an instance of overfitting can still be observed. The magnitude of the problem has decreased thanks to the dropout and data augmentation layer. Let's attempt to address this once and for all by also incorporating L2 regularization into the dense layer.

In [ ]:
from keras.regularizers import l2

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
])

inputs = keras.Input(shape=(64, 64, 3))
x = data_augmentation(inputs)
x = inputs
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256, kernel_regularizer=l2(0.01))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
model.summary()

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
    ),
    keras.callbacks.ModelCheckpoint(
        #'/content/drive/Shareddrives/Giaquinta_Pasqualetti/Models/vgg16_final.h5',
        '/content/vgg16_final.h5',
        monitor='val_loss',
        mode='min',
        save_best_only=True,
    )
]

train, val, test = load_datasets(BATCH_SIZE = 64)
res = model.fit(train, epochs=15, validation_data=val, callbacks = callbacks_list)
train_performance(res)
show_res(model, test)
plot_multiclass_roc(model, test)

# Fine Tuning
While this might not be the best idea (fine-tuning assumes that a model has been trained on similar data and with similar classes), we decided to try this approach anyway because we lack precise knowledge of the VGG16 model and because of the limited size of our dataset; we cannot definitively state that this will lead to worse results.

The model is composed of 19 layers as we can see below:

In [ ]:
#model = tf.keras.models.load_model('/content/drive/Shareddrives/Giaquinta_Pasqualetti/Models/vgg16_final.h5')
model = tf.keras.models.load_model('/content/vgg16_final.h5')

for i, layer in enumerate(model.get_layer('vgg16').layers):
    print(i, layer.name, layer.trainable)

In [ ]:
conv_base.summary()

In [ ]:
# Unfreeze every block
model.get_layer('vgg16').trainable = True

# Freeze every block except the last one
for layer in model.get_layer('vgg16').layers[0:15]:
    layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(model.get_layer('vgg16').layers):
    print(i, layer.name, layer.trainable)

In [ ]:
model.get_layer('vgg16').summary()

In [ ]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
    ),
    keras.callbacks.ModelCheckpoint(
        #'/content/drive/Shareddrives/Giaquinta_Pasqualetti/Models/vgg16_finetuned.h5',
        '/content/vgg16_finetuned.h5',
        monitor='val_loss',
        mode='min',
        save_best_only=True,
    )
]

train, val, test = load_datasets(BATCH_SIZE = 64)
res = model.fit(train, epochs=15, validation_data=val, callbacks = callbacks_list)
train_performance(res)
show_res(model, test)
plot_multiclass_roc(model, test)

Surely an unstable and unreliable model, fine tuning is not a good idea